In [11]:
import os
import pandas as pd
import numpy as np
import copy
import csv
import zipfile
from datetime import date
from  dateutil.relativedelta import relativedelta as rd
from math import isnan

In [12]:
cn_dict = np.load("data/cn_dict.npy",allow_pickle='TRUE').item()
ad_dict = np.load("data/ad_dict.npy",allow_pickle='TRUE').item()
smci_dict = np.load("data/smci_dict.npy",allow_pickle='TRUE').item()
pmci_dict = np.load("data/pmci_dict.npy",allow_pickle='TRUE').item()
mci_dict = np.load("data/mci_dict.npy",allow_pickle='TRUE').item()

In [13]:
IMAGES = pd.read_csv("data/Big_cohort_MRI_Images_25Oct2024.csv")
IMAGES[0:10]

,image_id,subject_id,mri_visit,mri_date,mri_description,mri_type,mri_weighting,mri_sequence,mri_thickness,mri_te,...,mri_flip_angle,mri_acq_plane,mri_width,mri_height,mri_n_images,mri_pixel_x,mri_pixel_y,mri_mfr,mri_mfr_model,mri_field_str
0,70846,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
1,70847,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
2,70848,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,SAGITTAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
3,70849,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
4,70850,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
5,70851,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,SAGITTAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
6,70852,037_S_1421,sc,2007-08-27,FSE PD/T2,2D,PD,SE,3.0,10.184000,...,90,AXIAL,256.0,256.0,48,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
7,70853,037_S_1421,sc,2007-08-27,FSE PD/T2,2D,T2,SE,3.0,101.839996,...,90,AXIAL,256.0,256.0,48,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
8,70854,037_S_1421,sc,2007-08-27,MP-RAGE REPEAT,3D,T1,RM,1.2,4.092000,...,8,SAGITTAL,256.0,256.0,180,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
9,70855,037_S_1421,sc,2007-08-27,MP-RAGE,3D,T1,RM,1.2,4.092000,...,8,SAGITTAL,256.0,256.0,180,0.93750,0.93750,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5


**Dict where patient id's are keys and the values are every single image with correspinding visit code.**

In [14]:
image_dict = {}

for index, row in IMAGES.iterrows():

    #Add filters to decide what images to include.
    if row['mri_type'] == '3D' and ('RAGE' in row['mri_description']) and ('REPEAT' not in row['mri_description']) and ('Repeat' not in row['mri_description']):
        
        if row['subject_id'] in image_dict:
            image_dict[row['subject_id']].append([row['mri_date'], row['mri_visit'], row['image_id']])
        else:
            image_dict[row['subject_id']] = [[row['mri_date'], row['mri_visit'], row['image_id']]]

In [23]:
print(image_dict)

{'037_S_1421': [['2007-08-27', 'sc', 70855], ['2008-02-20', 'm06', 91146], ['2008-09-17', 'm12', 118125], ['2009-05-13', 'm18', 143718], ['2010-09-24', 'm36', 193604]], '127_S_1427': [['2007-08-20', 'sc', 69355], ['2008-02-19', 'm06', 91511], ['2008-09-02', 'm12', 117511], ['2009-03-12', 'm18', 139006], ['2009-08-26', 'm24', 153734], ['2010-08-19', 'm36', 191032]], '027_S_1213': [['2007-01-19', 'sc', 37393], ['2007-07-20', 'm06', 61166], ['2008-01-16', 'm12', 87709], ['2008-07-28', 'm18', 114739], ['2008-07-28', 'm18', 114740], ['2009-01-26', 'm24', 134499]], '029_S_1218': [['2007-01-23', 'sc', 37373], ['2008-01-22', 'm12', 88197], ['2008-08-15', 'm18', 116268], ['2009-03-13', 'm24', 139184], ['2010-02-02', 'm36', 165706], ['2011-02-24', 'm48', 220190]], '037_S_1225': [['2007-01-24', 'sc', 37364], ['2007-02-09', 'bl', 38821], ['2007-08-20', 'm06', 69174], ['2007-08-20', 'm06', 69231], ['2008-02-11', 'm12', 90074], ['2008-02-11', 'm12', 90079], ['2008-09-08', 'm18', 117597], ['2008-09-0

In [24]:
images_for_download = list()

for key in image_dict:
    for visit in image_dict[key]:
        images_for_download.append(visit[2])

print(len(images_for_download))
print(images_for_download)

13860
[70855, 91146, 118125, 143718, 193604, 69355, 91511, 117511, 139006, 153734, 191032, 37393, 61166, 87709, 114739, 114740, 134499, 37373, 88197, 116268, 139184, 165706, 220190, 37364, 38821, 69174, 69231, 90074, 90079, 117597, 117606, 37350, 66831, 37339, 88930, 135508, 135509, 37701, 70419, 95515, 136342, 167951, 240936, 240938, 313932, 313935, 37284, 67828, 103748, 119345, 137796, 170642, 37270, 38482, 71869, 71878, 91623, 91625, 91633, 136457, 136486, 166407, 664578, 955206, 1291745, 10348204, 139566, 139567, 37182, 64542, 37145, 38417, 68042, 70290, 90149, 89333, 135046, 135176, 37129, 71813, 90324, 116260, 135349, 37076, 71166, 106931, 137077, 37062, 70033, 100034, 136121, 166456, 225333, 225340, 37662, 69456, 105076, 36411, 37761, 69912, 69921, 91605, 91614, 135827, 135849, 166158, 36398, 69669, 92773, 118098, 137812, 166813, 36373, 36374, 36364, 67311, 87973, 116508, 116509, 135623, 135624, 36281, 68257, 88752, 166068, 36269, 66933, 89355, 36230, 36178, 38219, 62314, 62386,

In [117]:
archive = zipfile.ZipFile('/Volumes/Extreme SSD/Download/Download_collection_dataset.zip', 'r')
all_paths = archive.namelist()
relevant_paths = {}
image_path_dict = {}

for path in all_paths:
    split_path = path.split("/")
    new_path = str('/'+split_path[0]+'/'+split_path[1]+'/'+split_path[2]+'/'+split_path[3]+'/'+split_path[4])
    relevant_paths[split_path[4]] = new_path

print(len(relevant_paths))
    

379


In [ ]:
for key in relevant_paths:
    for image in images_for_download:
        image_str = str(image)
        if ('I'+image_str) == key:
            image_path_dict[image_str] = relevant_paths[key]
            images_for_download.remove(image)

print(len(image_path_dict))

379


In [119]:
print(len(images_for_download))

2


Code for linking images to their respective paths

In [120]:
with open('images_with_paths_file_9.csv','w') as f:
    w = csv.writer(f)
    w.writerows(image_path_dict.items())

np.save("images_with_paths_file_9.npy", image_path_dict)

In [ ]:
path = '/Volumes/Extreme SSD/Download'
image_path_dict = {}


for subdir, dirs, files in os.walk(path):
    subdir_str = subdir.split('/')
    if len(subdir_str) == 9:
        for image in images_for_download:
            image_str = str(image)
            if ('I'+image_str) == (subdir_str[8]):
                image_path_dict[image] = subdir
                images_for_download.remove(image)

/Volumes/Extreme SSD/Download
/Volumes/Extreme SSD/Download/mni_icbm152_nlin_sym_09c


**Function for combining patient dict with image dict**

In [19]:
def combine_patients_pictures(in_dict):  

    counter = 0
    dict_with_images = copy.deepcopy(in_dict)
    return_dict = {}

    for patient in dict_with_images:

        if patient in image_dict:

            return_dict[patient] = list()

            #if len(str(dict_with_images[patient][0][1])) > 3 and len(str(dict_with_images[patient][-1][1])) > 3:
                
                #first_date = [int(x) for x in dict_with_images[patient][0][1].split("-")]
                #last_date = [int(x) for x in dict_with_images[patient][-1][1].split("-")]

                #first_date = date(first_date[0], first_date[1], first_date[2])
                #last_date = date(last_date[0], last_date[1], last_date[2])

            first_date = dict_with_images[patient][0][1]
            last_date = dict_with_images[patient][-1][1]

            for visit in image_dict[patient]:

                image_date = [int(x) for x in visit[0].split("-")]

                image_date = date(image_date[0], image_date[1], image_date[2])

                if (first_date - rd(months=1)) < image_date < (last_date + rd(months=1)):

                    if patient in return_dict:
                        existed = False
                        for visit2 in return_dict[patient]:
                        
                            if visit2[1] == str(image_date):

                                existed = True
                                visit2[3].append(visit[2])
                                counter += 1
                                break

                        if not existed:
                            
                            visit = [visit[1], visit[0], dict_with_images[patient][0][2], [visit[2]]]

                            return_dict[patient].append(visit)

                            counter += 1

                    else:

                        counter += 1

                        visit = [visit[1], visit[0], dict_with_images[patient][0][2], [visit[2]]]

                        return_dict[patient].append(visit)

                    
    print(counter)
    return return_dict


In [20]:
cn_dict_with_images_timewindow = combine_patients_pictures(cn_dict)
ad_dict_with_images_timewindow = combine_patients_pictures(ad_dict)
smci_dict_with_images_timewindow = combine_patients_pictures(smci_dict)
pmci_dict_with_images_timewindow = combine_patients_pictures(pmci_dict)
mci_dict_with_images_timewindow = combine_patients_pictures(mci_dict)

print(cn_dict_with_images_timewindow['023_S_0031'])
print(ad_dict_with_images_timewindow['011_S_0003'])
print(smci_dict_with_images_timewindow['022_S_0004'])
print(pmci_dict_with_images_timewindow['023_S_0030'])

2665
1469
2683
1313
5333
[['sc', '2005-10-12', 1.0, [7899]], ['bl', '2005-10-28', 1.0, [8298]], ['m06', '2006-04-17', 1.0, [13535, 13561]], ['m12', '2006-10-30', 1.0, [28019, 28030]], ['m24', '2007-11-14', 1.0, [82191, 82254]], ['m36', '2008-12-02', 1.0, [128980, 129040]], ['m48', '2009-10-22', 1.0, [157584]], ['m60', '2010-12-01', 1.0, [207222]], ['v06', '2011-10-20', 1.0, [261879]], ['v11', '2012-09-20', 1.0, [336528]], ['v21', '2013-10-17', 1.0, [395003]], ['v41', '2016-01-26', 1.0, [614787]], ['init', '2018-04-17', 1.0, [987370]], ['y1', '2019-04-24', 1.0, [1158135]]]
[['sc', '2005-09-01', 3.0, [7055]], ['m06', '2006-03-13', 3.0, [16547]], ['m12', '2006-09-12', 3.0, [24696]], ['m24', '2007-09-12', 3.0, [73250]]]
[['m06', '2006-05-25', 2.0, [16102]], ['m12', '2006-11-28', 2.0, [30799]], ['m18', '2007-05-25', 2.0, [65893]], ['m36', '2008-12-09', 2.0, [130026]]]
[['bl', '2005-10-26', 2.0, [8248]], ['sc', '2005-10-10', 2.0, [7526]]]


**Checking for any patient overlap between datasets**

In [22]:
if set(cn_dict_with_images_timewindow).intersection(ad_dict_with_images_timewindow) or set(cn_dict_with_images_timewindow).intersection(smci_dict_with_images_timewindow) or set(cn_dict_with_images_timewindow).intersection(pmci_dict_with_images_timewindow) or set(ad_dict_with_images_timewindow).intersection(smci_dict_with_images_timewindow) or set(ad_dict_with_images_timewindow).intersection(pmci_dict_with_images_timewindow) or set(smci_dict_with_images_timewindow).intersection(pmci_dict_with_images_timewindow):
    print("Error. Patient overlap")

else:
    print("No patient overlap.")

    np.save("data/cn_dict_with_images_timewindow.npy", cn_dict_with_images_timewindow)
    np.save("data/ad_dict_with_images_timewindow.npy", ad_dict_with_images_timewindow) 
    np.save("data/smci_dict_with_images_timewindow.npy", smci_dict_with_images_timewindow) 
    np.save("data/pmci_dict_with_images_timewindow.npy", pmci_dict_with_images_timewindow)
    np.save("data/mci_dict_with_images_timewindow.npy", mci_dict_with_images_timewindow)

No patient overlap.


In [28]:
cn_dict_with_images_timewindow['023_S_0031'][0][3][0]

7899